### connect to SQLite and create database

In [ ]:
import json
import sqlite3

#connect to sqlite 
conn = sqlite3.connect('[db_name].db')
cursor = conn.cursor()

create_table = """
    CREATE TABLE IF NOT EXISTS [table_name] (
    id INTEGER PRIMARY KEY);
    """
cursor.execute(create_table)

filenames = [r"[file_1].json", 
            r"[file_2].json"]

#get default data representation
default = json.load(open(filenames[0], encoding= 'utf-8'))
default[0].keys() #get json keys

schema = default[0]
columns = ['ts', 'platform', 'ms_played','ip_addr', 'master_metadata_track_name', 'master_metadata_album_artist_name', 'master_metadata_album_album_name', 
           'spotify_track_uri', 'reason_start', 'reason_end', 'shuffle', 'skipped', 'offline', 'offline_timestamp']

column_dict = {}

for k, v in schema.items(): #access dict k,v pairs
        dtype = str if type(v) == bool else type(v) #set dtype to current value type
        if k in columns: #if key in columns list, add to column_dict
            #print(dtype)
            column_dict[k] = dtype

for i in column_dict: #add all columns in column dict to db
        j = column_dict[i]
        insert_cmd = "ALTER TABLE [table_name] ADD COLUMN \'%s\' \'%s\' " % (i, j)
        cursor.execute(insert_cmd)
        
try: #add source_file column
    cursor.execute("ALTER TABLE [table_name] ADD COLUMN source_file TEXT;")
    conn.commit()
except sqlite3.OperationalError:
    pass

conn.commit()

#check schema
cursor.execute("""
        SELECT sql
        FROM sqlite_schema
        WHERE name = '[table_name]'
        """)
cursor.fetchall()


In [ ]:
#add full data to db
for file in filenames:
    fn = file
    data = json.load(open(fn, encoding= 'utf-8'))
    num_records = len(data)

    #check if file already inserted
    cursor.execute("SELECT COUNT(*) FROM [table_name] WHERE source_file = :source_file", (fn,))
    existing_rows = cursor.fetchall()

    #if file already inserted, skip if full record added, delete if partial insert
    if existing_rows == num_records: 
        continue
    else:
        cursor.execute("""
        SELECT COUNT(*) FROM [table_name] 
        WHERE source_file = :source_file;""", (fn,))
        conn.commit()
    
    for i in data:
        cursor.execute("""
            INSERT INTO [table_name]
            (ts, platform, ms_played, ip_addr, master_metadata_track_name, master_metadata_album_artist_name, master_metadata_album_album_name, 
            spotify_track_uri, reason_start, reason_end, shuffle, skipped, offline, offline_timestamp, source_file) 
            VALUES(:ts, :platform, :ms_played, :ip_addr, :master_metadata_track_name, :master_metadata_album_artist_name, :master_metadata_album_album_name, 
            :spotify_track_uri, :reason_start, :reason_end, :shuffle, :skipped, :offline, :offline_timestamp, :source_file)
            """, 
            (i['ts'], i['platform'], i['ms_played'], i['ip_addr'], i['master_metadata_track_name'], i['master_metadata_album_artist_name'], i['master_metadata_album_album_name'], 
             i['spotify_track_uri'], i['reason_start'], i['reason_end'], i['shuffle'], i['skipped'], i['offline'], i['offline_timestamp'], fn)
                  )
    conn.commit()
    
    #check if added records is expected
    cursor.execute("""
        SELECT COUNT(ts) FROM [table_name] 
        WHERE source_file = :source_file;""", (fn,))
    
    num_rows = cursor.fetchall()
    print(f"expected: {num_records}, /n actual: {num_rows}")

conn.close()


### optional: database cleanup 

In [ ]:
#remame columns for readability
columns_to_rename = {
    "ts": "timestamp",
    "ms_played": "duration_ms",
    "master_metadata_track_name": "track_name",
    "master_metadata_album_artist_name": "artist",
    "master_metadata_album_album_name": "album",
    "spotify_track_uri": "track_uri"}

for old_name, new_name in columns_to_rename.items():
    cursor.execute("""
        ALTER TABLE [table_name]
        RENAME COLUMN %s TO %s 
        """ % (old_name, new_name)
        )

#drop ip address and null columns
columns_to_drop = ["ip_addr", "offline", "offline_timestamp"]

for column in columns_to_drop:
    cursor.execute("""
    ALTER TABLE [table_name]
    DROP COLUMN %s""" % (column)
    )

#get filename from path (remove identifying info)
prefix = 'Streaming_History_Audio'
params = (prefix, prefix)

change_source_file = f"""
    UPDATE [table_name]
    SET source_file = SUBSTR(source_file, 
    INSTR(source_file, ?))
    WHERE INSTR(source_file, ?) > 0;
    """
cursor.execute(change_source_file, params)

#check schema
cursor.execute("""
    SELECT sql 
    FROM sqlite_schema 
    WHERE name = '[table_name]';
    """)

cursor.fetchall()

#commit changes
conn.commit()

[('CREATE TABLE [table_name] (\n    id INTEGER PRIMARY KEY, "timestamp" \'<class \'str\'>\', \'platform\' \'<class \'str\'>\', "duration_ms" \'<class \'int\'>\', "track_name" \'<class \'str\'>\', "artist" \'<class \'str\'>\', "album" \'<class \'str\'>\', "track_uri" \'<class \'str\'>\', \'reason_start\' \'<class \'str\'>\', \'reason_end\' \'<class \'str\'>\', \'shuffle\' \'<class \'str\'>\', \'skipped\' \'<class \'str\'>\', source_file TEXT)',)]